In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
import os
import numpy as np
from math import floor

In [2]:
# Laden des vortrainierten Inception-v3-Modells
model = InceptionV3(weights="imagenet")

In [3]:
def prepareImages(img_path):
    
    img = image.load_img(img_path, target_size=(299,299)) # Bild auf 299x299 skalieren
    img_array = image.img_to_array(img) # Bild in ein numpy Array umwandeln
    img_array = np.expand_dims(img_array, axis=0) # Bild Array auf [1, 299, 299, 3] erweitern
    img_array = preprocess_input(img_array) # Bilddaten für das Model anpassen (automatisch)
    
    return img_array


In [4]:
# assign directory
directory = './samples/cifar10'
 
# Array zum Speichern der Bilder erstellen
data = np.ones((1,299,299,3))

# Über die Datein im Ordner iterieren
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    
    # Kontrolle ob der Pfad wirklich eine Datei ist
    if os.path.isfile(f):
        data = np.append(data, prepareImages(f), axis=0)

# Das erste Element des Speicher Arrays löschen (Da nur mit Einsen gefüllt)
data = np.delete(data, 0, 0)

preds = model.predict(data)
print(preds.shape)

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step  
(100, 1000)


In [21]:
def calculate_inception_score(data, pred, n_split=10):
    # Datenset in Gruppen unterteilen um stabilere Ergebnisse zu erhalten
    scores = list()
    n_part = floor(data.shape[0] / n_split)
    for i in range(n_split):
        
        # Bedingte Klassenverteilung aus Prediction auslesen (Für den Abschnitt)
        ix_start, ix_end = i * n_part, i * n_part + n_part
        p_yx = pred[ix_start:ix_end]
        
        # Marginale Klassenverteilung berechnen
        p_y = np.expand_dims(p_yx.mean(axis=0), 0)
        
        # Die KL Divergenz mit hilfe Logorithmischer Wahrscheinlichkeiten berechnen
        kl_d = p_yx * (np.log(p_yx) - np.log(p_y))       

        # KL Divergenz aufsummieren (pro Bild)
        sum_kl_d = kl_d.sum(axis=1)

        # Durchschnitt der KL Divergenzen Bilden (pro Gruppe)
        avg_kl_d = np.mean(sum_kl_d)       

        # Logorithmische Operation Rückgängig machen durch exponieren
        is_score = np.exp(avg_kl_d)

        # Inception Scores abspeichern
        scores.append(is_score)
    
    # Den Durchschnitt sowie die Standardabweichung der IS Scores der Bilder bilden
    is_avg, is_std = np.mean(scores), np.std(scores)
    return is_avg, is_std

In [22]:
is_avg, is_std = calculate_inception_score(data, preds)
print("Inception Score (AVG): " + str(is_avg) +" | Inception Score (STD): "+ str(is_std))

Inception Score (AVG): 1.5521443 | Inception Score (STD): 0.07612172
